# 02 - Baseline Model

## Setup

In [182]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import yaml

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
# pd.set_option('display.max_rows', None)

sns.set_style("darkgrid")

from IPython.display import display, Markdown
from pprint import pprint 

DEBUG = True
SEED = 12092000

In [183]:
DATASET = "train.csv"

import os, sys
COLAB = 'google.colab' in sys.modules
ROOT = "./"

if COLAB:
  from google.colab import drive
  if not os.path.isdir("/content/gdrive"):
    drive.mount("/content/gdrive")
    d = "/content/gdrive/MyDrive/datasets"
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
  ROOT = f"/content/gdrive/MyDrive/datasets/{DATASET.replace(' ','_')}/"
  if not os.path.isdir(ROOT): os.makedirs(ROOT)


def makedirs(d):
  if COLAB:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
  else:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d, mode=0o777, exist_ok=True)

for d in ['orig','data','output']: makedirs(d)

## Import Data

In [184]:
df_train = pd.read_pickle(ROOT+"data/train.pkl")
print(df_train.shape)
df_train.head()

(125803, 165)


,x000,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,x011,x012,x013,x014,x015,x016,x017,x018,x019,x020,x021,x022,x023,x024,x025,x026,x027,x028,x029,x030,x031,x032,x033,x034,x035,x036,x037,x038,x039,x040,x041,x042,x043,x044,x045,x046,x047,x048,x049,x050,x051,x052,x053,x054,x055,x056,x057,x058,x059,x060,x061,x062,x063,x064,x065,x066,x067,x068,x069,x070,x071,x072,x073,x074,x075,x076,x077,x078,x079,x080,x081,x082,x083,x084,x085,x086,x087,x088,x089,x090,x091,x092,x093,x094,x095,x096,x097,x098,x099,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145,x146,x147,x148,x149,x150,x151,x152,x153,x154,x155,x156,x157,x158,x159,x160,x161,x162,x163,Target
74077,1.141575,8,-4.359983,0.450825,0.845550,x005_000,3.007819,1,x008_000,0.850699,3.025945,x011_001,0.264184,0.622826,-0.919733,-0.072886,x016_048,1.254801,x018_000,-0.092430,-0.919733,3,0.344038,3.025945,0.545339,0.843092,3.237924,0.430913,0.236528,-2.063537,33,1.163704,0.797879,0.845550,x034_005,x035_014,3.237924,-6.688506,0.553086,0.430913,5,3.075214,0,0.618561,x044_003,5.760403,x046_044,0,-0.092430,2.712984,1.679671,-1.450752,0.620773,x053_001,0.575879,3.579433,0.505691,0.0,0.684463,1.254801,0.604527,0.160393,x062_009,0,-1.931227,0.834569,0.620773,-1.767640,1,0.684463,0,x071_194,x072_000,x073_019,x074_000,7,-4.449852,0.620807,0.115102,2,-0.382699,0.649372,x082_000,0.476814,0.216968,0.184741,0.151682,0.190724,3,-3.213441,2.642693,-3.213441,x092_004,0.618561,2.135107,-8.276485,1.517087,x097_002,1.115526,0.480386,1.141575,0,0.816572,0.483895,0.834569,0.280958,x106_009,x107_006,-0.488003,x109_001,x110_055,x111_000,0.236528,0.199391,0.155979,-2.063773,0.813007,0.596687,-0.791877,0.483895,-0.488003,0.0,0.399602,5,x124_244,x125_002,x126_001,0.797879,2.712984,x129_050,0.393239,-0.773984,x132_021,1,x134_000,2.667655,1.145426,x137_001,0,5.132649,0.649372,0.604527,0.816572,-0.670385,5.132649,5.760403,0.216968,-1.450752,0.151682,-8.276485,x150_001,-1.931227,135,0.596687,0.843092,1.679671,-1.767640,x157_006,33,0.444652,-0.382699,0.381588,-0.829724,x163_010,Class_5
49922,1.143751,0,-3.438526,0.805108,-1.619605,x005_000,-0.366589,0,x008_000,-0.875760,4.290102,x011_001,0.878541,0.613965,-0.074597,1.528265,x016_048,-3.035078,x018_000,2.293614,-0.074597,1,0.280777,4.290102,-8.714742,0.281836,0.602296,0.310266,0.308363,-0.289196,33,4.698847,0.721183,-1.619605,x034_010,x035_014,0.602296,-2.999018,0.238200,0.310266,7,-4.134411,0,0.289259,x044_003,-0.753635,x046_043,0,2.293614,-0.084513,NaN,-4.425191,-0.247047,x053_000,0.214749,7.793300,0.345678,0.0,0.984625,-3.035078,0.646445,0.898021,x062_011,0,11.079512,0.671892,-0.247047,-1.175019,1,0.984625,9,x071_123,x072_000,x073_002,x074_000,4,2.966075,0.420835,0.197124,1,0.773790,0.551325,x082_000,0.710899,-0.817111,-1.064741,1.722823,0.781980,3,1.796974,3.720548,1.796974,x092_001,0.289259,-1.150620,4.091824,-0.650828,x097_016,1.115781,0.356202,1.143751,1,-1.818711,0.278035,0.671892,0.311779,x106_002,x107_012,1.485849,x109_000,x110_057,x111_001,0.308363,0.719172,1.147369,0.927291,0.362872,0.549868,-0.235604,0.278035,1.485849,0.0,0.210574,11,x124_022,x125_002,x126_001,0.721183,-0.084513,x129_002,0.267381,4.433305,x132_038,1,x134_000,-0.667662,0.101721,x137_001,0,4.598879,0.551325,0.646445,-1.818711,-3.151749,4.598879,-0.753635,-0.817111,-4.425191,1.722823,4.091824,x150_001,11.079512,256,0.549868,0.281836,0.282718,-1.175019,x157_037,33,0.315624,0.773790,0.321262,-1.191636,x163_010,Class_5
25511,-1.636225,0,0.120206,0.828269,0.414759,x005_003,0.284236,0,x008_000,1.466211,3.070626,x011_000,0.819580,0.729508,1.250118,-0.398999,x016_004,0.288877,x018_000,2.313134,1.250118,3,0.830299,3.070626,-2.932911,0.807711,2.698524,0.564777,0.659052,1.711316,33,6.474173,0.594748,0.414759,x034_009,x035_014,2.698524,-0.078190,0.282701,0.564777,4,3.935656,0,0.280576,x044_003,-1.288236,x046_040,0,2.313134,-2.629

In [185]:
df_test = pd.read_pickle(ROOT+"data/test.pkl")
print(df_test.shape)
df_test.head()

(83869, 165)


,x000,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,x011,x012,x013,x014,x015,x016,x017,x018,x019,x020,x021,x022,x023,x024,x025,x026,x027,x028,x029,x030,x031,x032,x033,x034,x035,x036,x037,x038,x039,x040,x041,x042,x043,x044,x045,x046,x047,x048,x049,x050,x051,x052,x053,x054,x055,x056,x057,x058,x059,x060,x061,x062,x063,x064,x065,x066,x067,x068,x069,x070,x071,x072,x073,x074,x075,x076,x077,x078,x079,x080,x081,x082,x083,x084,x085,x086,x087,x088,x089,x090,x091,x092,x093,x094,x095,x096,x097,x098,x099,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145,x146,x147,x148,x149,x150,x151,x152,x153,x154,x155,x156,x157,x158,x159,x160,x161,x162,x163,Target
122232,-3.344156,4,2.172569,0.539517,-0.232310,x005_002,-0.347396,0,x008_001,-0.158356,3.660330,x011_000,0.295093,0.767444,0.349099,1.134375,x016_006,1.694801,x018_000,1.283826,0.349099,3,0.288078,3.660330,-0.576699,0.701344,1.845780,0.720627,0.424103,1.373123,33,6.051348,0.552332,-0.232310,x034_009,x035_011,1.845780,-2.733806,0.295020,0.720627,7,-4.827230,0,0.737022,x044_003,1.772232,x046_005,0,1.283826,1.702993,2.000092,5.646412,0.960414,x053_001,0.525748,-3.773189,0.347604,0.0,0.196037,1.694801,0.332869,0.471527,x062_004,3,1.544974,0.554083,0.960414,-0.834783,1,0.196037,0,x071_235,x072_000,x073_014,x074_000,4,2.506398,0.354329,0.119601,0,-1.432546,0.323790,x082_003,0.925484,-0.754708,-4.157989,1.993699,0.425550,1,4.000094,-1.020161,4.000094,x092_010,0.737022,-0.779117,-0.219093,0.004463,x097_002,-2.531398,0.496891,-3.344156,1,-3.164401,0.794238,0.554083,0.747685,x106_010,x107_012,0.642362,x109_001,x110_057,x111_000,0.424103,0.465203,-5.717978,-1.301262,0.346044,0.864574,1.441809,0.794238,0.642362,0.0,0.383641,7,x124_282,x125_002,x126_001,0.552332,1.702993,x129_015,0.354346,-1.348996,x132_039,2,x134_000,-2.655006,2.517596,x137_000,0,-2.593382,0.323790,0.332869,-3.164401,-0.690828,-2.593382,1.772232,-0.754708,5.646412,1.993699,-0.219093,x150_003,1.544974,23,0.864574,0.701344,2.000092,-0.834783,x157_018,33,0.707962,-1.432546,0.887848,-0.079848,x163_010,Class_3
170143,3.231669,0,5.213101,0.319086,0.563226,x005_002,-1.936628,1,x008_000,-0.485216,1.174234,x011_000,0.934481,0.282337,1.330622,-1.395256,x016_013,0.495893,x018_000,0.398433,1.330622,1,0.319295,1.174234,0.802635,0.913934,2.563651,0.458742,0.342443,-1.454100,33,2.706768,0.235929,0.563226,x034_009,x035_004,2.563651,2.295860,0.512712,0.458742,6,1.656521,1,0.071356,x044_003,-2.010164,x046_050,0,0.398433,-2.289960,5.413164,0.230010,2.035408,x053_001,0.805215,3.957178,0.701032,0.0,0.788725,0.495893,0.215370,0.362263,x062_009,0,1.319552,0.347096,2.035408,1.569314,2,0.788725,14,x071_090,x072_002,x073_001,x074_000,7,6.514606,0.619366,0.764356,1,0.633420,0.524824,x082_003,0.899786,0.676550,-0.612299,-0.554123,0.355339,3,-2.173246,-1.131016,-2.173246,x092_006,0.071356,-0.381436,1.670339,-1.074664,x097_004,-2.825852,0.509803,3.231669,0,-5.910125,0.342076,0.347096,0.617534,x106_002,x107_006,2.737555,x109_001,x110_057,x111_000,0.342443,0.971688,-1.585377,-2.785407,0.762415,0.431396,-1.560924,0.342076,2.737555,0.0,0.679771,8,x124_018,x125_002,x126_001,0.235929,-2.289960,x129_003,0.595006,1.126768,x132_048,3,x134_000,-0.307294,0.159046,x137_001,0,1.062362,0.524824,0.215370,-5.910125,-0.625540,1.062362,-2.010164,0.676550,0.230010,-0.554123,1.670339,x150_003,1.319552,271,0.431396,0.913934,5.413164,1.569314,x157_029,33,0.312975,0.633420,0.319985,-0.152520,x163_010,Class_1
196172,0.796761,0,0.314170,0.501443,0.485478,x005_000,-0.231908,1,x008_000,-1.218573,-1.200363,x011_000,0.300099,0.423472,-0.809290,-0.594823,x016_013,1.040214,x018_000,-0.286775,-0.809290,3,0.577801,-1.200363,2.966585,0.428781,0.015534,0.361838,0.887167,-0.013237,33,4.990464,0.778387,0.485478,x034_009,x035_014,0.015534,-0.337644,0.339875,0.361838,7,1.114214,0,0.075828,x044_003,4.480100,x046_011,5,-0.286775,

In [186]:
df_score = pd.read_pickle(ROOT+"data/score.pkl")
print(df_score.shape)
df_score.head()

(90000, 164)


,x000,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,x011,x012,x013,x014,x015,x016,x017,x018,x019,x020,x021,x022,x023,x024,x025,x026,x027,x028,x029,x030,x031,x032,x033,x034,x035,x036,x037,x038,x039,x040,x041,x042,x043,x044,x045,x046,x047,x048,x049,x050,x051,x052,x053,x054,x055,x056,x057,x058,x059,x060,x061,x062,x063,x064,x065,x066,x067,x068,x069,x070,x071,x072,x073,x074,x075,x076,x077,x078,x079,x080,x081,x082,x083,x084,x085,x086,x087,x088,x089,x090,x091,x092,x093,x094,x095,x096,x097,x098,x099,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145,x146,x147,x148,x149,x150,x151,x152,x153,x154,x155,x156,x157,x158,x159,x160,x161,x162,x163
0,2.319538,0,5.020483,0.448529,-0.852546,x005_000,-0.998906,0,x008_000,1.288086,4.673962,x011_001,0.289747,0.284042,0.572018,-1.336891,x016_006,-1.137960,x018_002,2.442883,0.572018,3,0.298722,4.673962,1.958812,0.282664,4.855930,0.290681,0.480461,-0.091239,33,-0.047759,0.288419,-0.852546,x034_011,x035_014,4.855930,2.191471,0.302269,0.290681,4,-2.573493,1,0.789406,x044_000,-2.778244,x046_011,0,2.442883,-3.219411,-1.538807,0.634541,0.358364,x053_001,0.410863,4.025498,0.314780,1.0,0.885479,-1.137960,0.647797,0.475543,x062_001,0,-0.084794,0.360460,0.358364,-1.810576,1,0.885479,3,x071_018,x072_000,x073_002,x074_000,5,4.516900,0.735640,0.477497,0,0.084926,0.510748,x082_000,0.310362,-3.295310,2.907393,0.377336,0.680271,1,-1.684449,-1.089001,-1.684449,x092_004,0.789406,-1.758931,-0.920925,-1.816968,x097_002,-3.853039,0.335260,2.319538,1,1.210150,0.823916,0.360460,0.422275,x106_010,x107_006,-2.845971,x109_001,x110_059,x111_000,0.480461,0.221567,1.620115,-3.280507,0.619155,0.612774,-1.956931,0.823916,-2.845971,0.0,0.431122,12,x124_022,x125_000,x126_000,0.288419,-3.219411,x129_023,0.153943,3.630768,x132_009,1,x134_000,-0.712996,1.072984,x137_001,0,0.708755,0.510748,0.647797,1.210150,4.905962,0.708755,-2.778244,-3.295310,0.634541,0.377336,-0.920925,x150_003,-0.084794,159,0.612774,0.282664,-1.538807,-1.810576,x157_051,33,0.307058,0.084926,0.279340,0.626329,x163_007
1,0.596632,0,6.648641,0.508439,0.967977,x005_002,-2.897833,1,x008_001,0.376745,-4.264011,x011_000,0.365204,0.285102,0.219571,-0.313663,x016_048,0.629423,x018_000,-1.675292,0.219571,3,0.279568,-4.264011,-1.111490,0.283594,-0.828395,0.351714,0.625054,1.848905,2,-2.033200,0.805128,0.967977,x034_001,x035_014,-0.828395,-0.321451,0.142981,0.351714,5,-0.629202,1,0.119783,x044_005,-3.858438,x046_005,0,-1.675292,2.657546,0.117252,-1.301271,0.605371,x053_001,0.171433,-2.538732,0.287723,0.0,0.904710,0.629423,0.274973,0.463415,x062_012,2,-0.470625,0.491269,0.605371,0.881008,3,0.904710,0,x071_157,x072_001,x073_002,x074_000,5,1.517297,0.490456,0.466524,0,0.583530,0.533045,x082_003,0.685248,-0.248232,-5.086252,2.109288,0.277475,3,1.862004,-2.096803,1.862004,x092_004,0.119783,0.103561,-0.830837,-0.850213,x097_002,-4.353762,0.166441,0.596632,1,0.074920,0.147904,0.491269,0.672344,x106_009,x107_002,-1.922535,x109_001,x110_057,x111_000,0.625054,0.317563,2.387910,-2.485752,0.419489,0.910860,-1.030031,0.147904,-1.922535,0.0,0.828945,7,x124_022,x125_002,x126_001,0.805128,2.657546,x129_010,0.163536,-3.465025,x132_041,1,x134_000,-1.683605,0.320888,x137_001,0,-5.786395,0.533045,0.274973,0.074920,1.152160,-5.786395,-3.858438,-0.248232,-1.301271,2.109288,-0.830837,x150_003,-0.470625,69,0.910860,0.283594,0.117252,0.881008,x157_019,33,0.303058,0.583530,0.483201,0.309898,x163_010
2,1.181403,7,-0.278018,0.269135,0.470354,x005_000,-1.125396,1,x008_000,-0.435548,4.971689,x011_001,0.922037,0.876832,0.792961,-0.719348,x016_048,1.056223,x018_000,1.310970,0.792961,3,0.651189,4.971689,-2.513337,0.283730,-2.852422,0.122145,0.312218,0.711132,33,-3.136320,0.794412,0.470354,x034_009,x035_014,-2.852422,-4.229822,0.297356,0.122145,4,-1.342726,1,0.149601,x044_005,1.935087,x046_041,0,1.310970,-0.849471,0.256365,-4.095485

## Generate Submission Files

In [187]:
target_labels = sorted(df_train['Target'].unique())
pred = np.array([[0.1, 0.3, 0.33, 0.27, 0.1]] * 90000)
pd.DataFrame(pred, columns=target_labels)

def submission(y_pred_prob, name=f"submission-{pd.to_datetime('now', utc=True).strftime('%Y%m%d%H%M%S')}"):
    df_sub = pd.DataFrame(y_pred_prob, columns=target_labels)
    df_sub.index.name = 'id'
    df_sub.to_csv(ROOT+f'output/{name}.csv', index=True)
    print(f"Saved ({df_sub.shape[0]} rows) to: {ROOT}output/{name}.csv")

## Baseline Model

* treat NA - drop or impute? Impute is faster != correct

In [188]:
# All missing values
na_vals = df_train.isna().sum().sum()
print(f"Missing values: {na_vals}")

Missing values: 9974


In [189]:
# All rows with at least one NaN
nan_rows = (df_train.isna().sum(axis=1)>0).sum()
print(f"Rows with at least one NaN: {nan_rows} ({nan_rows/df_train.shape[0]:.2%})")

Rows with at least one NaN: 9639 (7.66%)


In [190]:
# Really stupid but fast - treat all NA as 0
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)
df_score = df_score.fillna(0)

### Dont do above !

* Numerical populate by mean
* Categorical populate by mode
* If skewed populate by median

target = 'Target'
features = df.columns.drop(target)

In [191]:
target = 'Target'
features = df_train.columns.drop(target)

In [192]:
features = list(df_train.select_dtypes(["int", "float"]).columns)
df_train.dtypes.value_counts()

float64    112
object      34
int64       19
dtype: int64

In [193]:
df_train.select_dtypes("int").nunique().sort_values()

x007      2
x138      2
x101      2
x021      4
x133      4
x068      4
x088      4
x042     13
x079     13
x123     15
x001     19
x070     19
x063     19
x047     19
x040     20
x075     20
x030     84
x158     84
x152    286
dtype: int64

In [194]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(df_train[features])

X_train = ss.transform(df_train[features])
X_test = ss.transform(df_test[features])
X_score = ss.transform(df_score[features])

In [195]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

y_train = df_train[target]
y_test = df_test[target]

In [196]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=SEED, max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=12092000)

In [197]:
from sklearn.metrics import confusion_matrix, classification_report, log_loss

y_pred = model.predict(X_train)
y_pred_prob = model.predict_proba(X_train)

print(confusion_matrix(y_train, y_pred))
print()
print(classification_report(y_train, y_pred))
print()
print(f"Log Loss: {log_loss(y_train, y_pred_prob):.4f}")

[[19491  2797  1409  1009  3828]
 [ 6005  6576  1322   684 13584]
 [11486  2482  1714   357  7534]
 [ 9256  1069  1391  1333  2308]
 [ 4413  6146  1642   223 17744]]

              precision    recall  f1-score   support

     Class_1       0.38      0.68      0.49     28534
     Class_2       0.34      0.23      0.28     28171
     Class_3       0.23      0.07      0.11     23573
     Class_4       0.37      0.09      0.14     15357
     Class_5       0.39      0.59      0.47     30168

    accuracy                           0.37    125803
   macro avg       0.34      0.33      0.30    125803
weighted avg       0.35      0.37      0.33    125803


Log Loss: 1.4334


In [198]:
y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)

print(confusion_matrix(y_test, y_pred))
print()
print(classification_report(y_test, y_pred))
print()
print(f"Log Loss: {log_loss(y_test, y_pred_prob):.4f}")

[[13022  1831   931   642  2596]
 [ 4074  4286   897   429  9095]
 [ 7689  1711  1124   276  4915]
 [ 6132   716   940   870  1580]
 [ 2974  4205  1143   135 11656]]

              precision    recall  f1-score   support

     Class_1       0.38      0.68      0.49     19022
     Class_2       0.34      0.23      0.27     18781
     Class_3       0.22      0.07      0.11     15715
     Class_4       0.37      0.08      0.14     10238
     Class_5       0.39      0.58      0.47     20113

    accuracy                           0.37     83869
   macro avg       0.34      0.33      0.30     83869
weighted avg       0.34      0.37      0.32     83869


Log Loss: 1.4349


In [199]:
y_pred_prob = model.predict_proba(X_score)
print(y_pred_prob.shape)
submission(y_pred_prob, "LR_num_features_only")

(90000, 5)
Saved (90000 rows) to: ./output/LR_num_features_only.csv
